# Демонстрация обучения модели

In [9]:
from SimpleDataset import SimpleDataset
from NewBacktest import NewBacktest
import pandas as pd
from TrainModel import TrainModel

In [10]:
timestamp = "10m" # Гранулярность данных
ticker = "WUSH" # Акция

## Зададим признаки для обучения модели

In [11]:
features = {'lags': {'features': ['open', 'close', 'target'], # Выбираем параметры и размеры лагов 
                     'period': [1, 2, 3]}, 

            'cma': {'features': ['open', 'close', 'volume']}, # Параметры для построение CMA (без таргета, чтобы избежать лика)

            'sma': {'features': ['open', 'close', 'volume'], # Параметры для построения SMA (без таргета, чтобы избежать лика)
                    'period': [2, 3, 4, 10, 100]},

            'ema': {'features': ['open', 'close', 'volume'], # Параметры для построения EMA (без таргета, чтобы избежать лика)
                    'period': [2, 3, 4, 10, 100]},

            'green_candles_ratio': {'period': [2, 5, 7, 14]}, # Соотношение зеленых свечей и всех свечей на разных периодах

            'red_candles_ratio': {'period': [2, 5, 7, 14]}, # Соотношение красынх свечей и всех свечей на разных периодаъ

            'rsi': {'period': [5, 10, 15, 20]},  # Параметры для построения RSI на нескольких периодах

            'macd': {'period': [(12, 26)]}, # только (12, 26) # Парамтеры для построения macd

            'bollinger': {'period': 10, 'degree_of_lift': 2}, # Параметры для линий Боллинджера

            'time_features': {'month':True, # Признаки времени
                              'week':True,
                              'day_of_month':True,
                              'day_of_week':True,
                              'hour':True,
                              'minute': True},
            'model': 'catboost'} # Выбираем модель для обучения

## Обучение модели

In [12]:
learner = TrainModel(
    features=features,
    ticker=ticker,
    timeframe=timestamp,
    model_id="test_model_id",
    notebook=True,
)

In [13]:
outp = learner.train()

train
Period from: 2023-03-23 15:49:59 to 2023-10-19 12:39:59
Target mean: 2.1e-05
Target std: 0.004
Target max: 0.075944
Target min: -0.053087
Shape of sample: (7920, 70)
--------------------------------------------------------------------
valid
Period from: 2023-10-19 12:49:59 to 2023-11-14 15:39:59
Target mean: 2.7e-05
Target std: 0.003002
Target max: 0.034584
Target min: -0.035268
Shape of sample: (990, 70)
--------------------------------------------------------------------
hold-out for backtest
Period from: 2023-11-14 15:49:59 to 2023-12-08 18:39:59
Target mean: -6.2e-05
Target std: 0.00391
Target max: 0.022147
Target min: -0.061096
Shape of sample: (990, 70)
--------------------------------------------------------------------
CatBoost RMSE score on validation set: 0.0030001332559561247
Threshold from Xtest 0.00016694409573937244


## Backtest на последних свечах

In [14]:
PART_OF_SUM_FOR_BUY = 0.1
PERCENT_TRAILING = 6
START_SUM = 100_000
PERC_FOR_STOP = 1.2

In [15]:
backtest = NewBacktest(
    type="ml_model",
    model_id="test_model_id",
    ticker=ticker,
    timestamp=timestamp,
    part_of_sum_for_buy=PART_OF_SUM_FOR_BUY,
    percent_trailing=PERCENT_TRAILING,
    start_sum=START_SUM,
    perc_for_stop=PERC_FOR_STOP,
    model_features=outp,
    notebook=True,
)

In [16]:
backtest.do_backtest(candles=990)

Start                     2023-11-14 15:50:00
End                       2023-12-08 18:40:00
Duration                     24 days 02:50:00
Exposure Time [%]                    99.79798
Equity Final [$]                100548.252311
Equity Peak [$]                 103802.393753
Return [%]                           0.548252
Buy & Hold Return [%]               -6.468328
Return (Ann.) [%]                    5.771371
Volatility (Ann.) [%]                31.14398
Sharpe Ratio                         0.185313
Sortino Ratio                         0.37542
Calmar Ratio                         0.772081
Max. Drawdown [%]                   -7.475089
Avg. Drawdown [%]                    -2.90175
Max. Drawdown Duration       16 days 22:10:00
Avg. Drawdown Duration        4 days 00:20:00
# Trades                                  797
Win Rate [%]                        44.165621
Best Trade [%]                        9.64187
Worst Trade [%]                     -4.366173
Avg. Trade [%]                    